# Function Download
filename = Directory +"/"+ filename+".jpg" #อันนี้รองรับการสร้าง Folder แล้วยัดไปเรื่อยๆ 
ระวังเรื่องการใช้งาน Directory นะ

In [1]:
#!/usr/bin/python

# Note to Kagglers: This script will not run directly in Kaggle kernels. You
# need to download it and run it on your local machine.

# Downloads images from the Google Landmarks dataset using multiple threads.
# Images that already exist will not be downloaded again, so the script can
# resume a partially completed download. All images will be saved in the JPG
# format with 90% compression quality.

import sys, os, multiprocessing, csv
from PIL import Image
from io import BytesIO
from urllib.request import urlopen
import time
import os

def ParseData(data_file):
    csvfile = open(data_file, 'r')
    csvreader = csv.reader(csvfile)
    key_url_list = [line[:3] for line in csvreader]
    return key_url_list[1:]  # Chop off header

import urllib.request

def download_image(url,filename,Directory):
#*************************************** ระวังผิด directory นะ set ก่อนด้วย ****************************
        filename = Directory +"/"+ filename+".jpg" #อันนี้รองรับการสร้าง Folder แล้วยัดไปเรื่อยๆ
#*************************************** ระวังผิด directory นะ set ก่อนด้วย ****************************
        if os.path.exists(filename):
            print('Image %s already exists. Skipping download.' % filename)
            return
        try:
            try :
                response = urlopen(url)
                image_data = response.read()
            except : 
                print("404 NOT FOUND")
        except:
            print('Warning: Could not download image %s from %s' % (key, url))
            return
        try:
            pil_image = Image.open(BytesIO(image_data))
        except:
            print('Warning: Failed to parse image %s' % key)
            return
        urllib.request.urlretrieve(url,filename)     


# Create downloadable file
fomat data เป็น Csv. 3 colurm จะเป็นผมให้มันเป็น แรก 3 line ในคำสั่ง ParseData(path)

In [2]:
csv_download = ParseData('landmark_recognition-master/data/data.csv')

In [3]:
#Play safe for index_boundary tranform list to dict 
label_to_category = ParseData('landmark_recognition-master/data/label_to_category.csv')
label_to_category_dict = {}

for el_lable in  label_to_category :
    label_to_category_dict[el_lable[2]] = [el_lable[0],el_lable[1]]
print(label_to_category_dict["0"]) 

['192931', 'york_river_state_park']


# Data split train-test

In [4]:
import numpy as np 
import pandas as pd
len(csv_download)

91618

In [5]:
#Split data 
#ตัวของข้อมูลนั้นทุกตัวคือคนละ Class 
#Expect ต้องแยกได้ 7 Folder

lst_raw_order = [] #L คือ List ของภาพที่เป็นคนละ Class
for i in range(len(csv_download)) :  # Loop นี้จะแยกและหาตำแหน่ง สุดท้าย ของ Class แต่ละ Class
    if i == len(csv_download)-1 :
        lst_raw_order.append(int(csv_download[i][0]))
        break
    if (csv_download[i][2] != csv_download[i+1][2]) :
        #print(i)
        lst_raw_order.append(int(csv_download[i][0]))
c = 0
# for j in lst_data : 
#     print(c,j,"\n")
#     c += 1 

In [6]:
raw_data = np.asarray(csv_download)
lst_raw_data_split = []
for i in range(100) : 
    if ( i == 0 ) : 
        lst_raw_data_split.append(raw_data[:lst_raw_order[i]])
    else : 
        lst_raw_data_split.append(raw_data[lst_raw_order[i-1]:lst_raw_order[i]] )
print(len(lst_raw_data_split))
#test Miss data in list
for j in range(100) : 
    for el_lst in range(1,len(lst_raw_data_split[i])) : 
        if (lst_raw_data_split[i][el_lst][2] != lst_raw_data_split[i][el_lst][2] ) : 
            print("error_detect",lst_raw_data_split[1][el_lst])

100


In [7]:
train_set = []
test_set  = []
var_set   = []
Data = []
for el_class in lst_raw_data_split :
    el_class = pd.DataFrame(el_class)
    train, validate, test = np.split(el_class.sample(frac=1), [int(.8*len(el_class)), int(.9*len(el_class))])
    train_set.append(train)
    test_set.append(test)
    var_set.append(validate)
    Data.append(list([train,test,validate]))

In [8]:
Data[0] #Validation

[         0                                                  1  2
 261    261  https://upload.wikimedia.org/wikipedia/commons...  0
 975    975  https://upload.wikimedia.org/wikipedia/commons...  0
 42      42  https://upload.wikimedia.org/wikipedia/commons...  0
 1525  1525  https://upload.wikimedia.org/wikipedia/commons...  0
 138    138  https://upload.wikimedia.org/wikipedia/commons...  0
 2205  2205  https://upload.wikimedia.org/wikipedia/commons...  0
 49      49  https://upload.wikimedia.org/wikipedia/commons...  0
 1659  1659  https://upload.wikimedia.org/wikipedia/commons...  0
 287    287  https://upload.wikimedia.org/wikipedia/commons...  0
 1751  1751  https://upload.wikimedia.org/wikipedia/commons...  0
 1453  1453  https://upload.wikimedia.org/wikipedia/commons...  0
 1765  1765  https://upload.wikimedia.org/wikipedia/commons...  0
 1317  1317  https://upload.wikimedia.org/wikipedia/commons...  0
 1458  1458  https://upload.wikimedia.org/wikipedia/commons...  0
 2369  236

# Start Download

แปลงเป็น Function ที่สามารถเรียกค่า ของ Directory และจัดการไฟล์

In [9]:
label_to_category_dict

{'0': ['192931', 'york_river_state_park'],
 '1': ['186080', 'osaka_prefectural_flower_garden'],
 '2': ['19605', 'saiful_muluk_lake'],
 '3': ['171683', 'lok_virsa_museum,_islamabad'],
 '4': ['189811', 'lake_como'],
 '5': ['86869', 'mount_rainier_national_park'],
 '6': ['90021', 'rohtas_fort'],
 '7': ['62798', 'enchanted_floral_gardens_of_kula'],
 '8': ['181631', 'leesylvania_state_park'],
 '9': ['10618', 'botanical_garden_jevremovac'],
 '10': ['165596', 'edinburgh_castle'],
 '11': ['165900', 'mount_arapiles'],
 '12': ['145015', 'texas_state_library_and_archives_commission'],
 '13': ['171772', 'university_of_chicago_library'],
 '14': ['67416', 'manassas_national_battlefield_park'],
 '15': ['177194', 'royal_national_park'],
 '16': ['107750', 'chippokes_plantation_state_park'],
 '17': ['137203', 'port_of_casablanca'],
 '18': ['73300', 'iguazu_falls'],
 '19': ['149980', 'khotyn_fortress'],
 '20': ['176018', 'hayravank_monastery'],
 '21': ['184313', 'kahanu_garden'],
 '22': ['16658', 'glasne

In [11]:

#*************************************** ระวังผิด directory นะ set ก่อนด้วย ****************************
 #Use disc directory
#*************************************** ระวังผิด directory นะ set ก่อนด้วย ****************************
def Download_image(start,end) :   
    start_time = time.time()
     # label_to_category_dict[str(temp_dir)][1] จะเรียก ชื่อ Landmark จาก Dict มา

        for j in range(start,end): 
            temp_dir = j 
            directory = label_to_category_dict[str(temp_dir)][1]
            print("Do CLass 1")
                for i in range(3) :
                    if ( i == 0  ) : 
                        base_directory = "D:/Train"
                        if not (os.path.exists(base_directory)) :
                            os.makedirs(base_directory)
                    elif (i == 1 ) : 
                        base_directory = "D:/Test"
                        if not (os.path.exists(base_directory)) :
                            os.makedirs(base_directory)
                    elif (i == 2 ) : 
                        base_directory = "D:/Validate"
                        if not (os.path.exists(base_directory)) :
                            os.makedirs(base_directory)
                    else : 
                        print("error create Test-Train-Valiadate")
                        break
                    for k in range(len(Data[j][i]))  :
                        try :
                            if os.path.exists(base_directory +'/'+ directory): #ใส่รูปไปเรื่อยๆ จนกว่าจะไปเจอ Directory 
                                #print('Image %s already exists. Skipping download.' % base_directory + directory)
                                download_image(Data[j][i][1].iloc[k],Data[j][i][0].iloc[k],base_directory+'/'+ directory)
                            else : 
                                os.makedirs(base_directory +'/'+ directory) #เมื่อต้องสร้างไฟล์ใหม่ ต้องมีการใส่รูป
                                download_image(Data[j][i][1].iloc[k],Data[j][i][0].iloc[k],base_directory+'/'+ directory)
                            if ( k % 150 == 0) : 
                                print("Pass",str(k))
                                end_time = time.time()
                                print(end_time-start_time)
                                start_time = time.time()

                        except KeyboardInterrupt:
                            return

                        except :
                            print("error")
        return True
